In [1]:
import pyspark as p
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark import SparkContext
from pyspark.sql.types import StructType, StructField, IntegerType, StringType
from pyspark.sql.window import Window

In [2]:
sc = SparkContext(appName="Read CSV as RDD Example")
spark = SparkSession(sc)
file_path = 'data/adult/adult.data'

24/11/12 12:21:10 WARN Utils: Your hostname, fynn-ThinkPad-L13-Yoga-Gen-2 resolves to a loopback address: 127.0.1.1; using 147.142.158.159 instead (on interface wlp0s20f3)
24/11/12 12:21:10 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/11/12 12:21:11 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [53]:
from pyspark.sql import functions as F

rdd = sc.textFile(file_path)
rdd = rdd.map(lambda x: [element.strip() for element in x.split(', ')])
rdd.take(5)
question_mark_values = rdd.filter(lambda x: '?' in x).collect()
print("QUESTION MARKT VALLUES", question_mark_values)
rdd = rdd.map(lambda x: [None if element == '?' else element for element in x])
# Cast the strings with only numbers to integers
rdd = rdd.map(lambda x: [int(element) if element and element.isdigit() else element for element in x])
rdd.take(5)



QUESTION MARKT VALLUES [['40', 'Private', '121772', 'Assoc-voc', '11', 'Married-civ-spouse', 'Craft-repair', 'Husband', 'Asian-Pac-Islander', 'Male', '0', '0', '40', '?', '>50K'], ['54', '?', '180211', 'Some-college', '10', 'Married-civ-spouse', '?', 'Husband', 'Asian-Pac-Islander', 'Male', '0', '0', '60', 'South', '>50K'], ['31', 'Private', '84154', 'Some-college', '10', 'Married-civ-spouse', 'Sales', 'Husband', 'White', 'Male', '0', '0', '38', '?', '>50K'], ['18', 'Private', '226956', 'HS-grad', '9', 'Never-married', 'Other-service', 'Own-child', 'White', 'Female', '0', '0', '30', '?', '<=50K'], ['32', '?', '293936', '7th-8th', '4', 'Married-spouse-absent', '?', 'Not-in-family', 'White', 'Male', '0', '0', '40', '?', '<=50K'], ['25', '?', '200681', 'Some-college', '10', 'Never-married', '?', 'Own-child', 'White', 'Male', '0', '0', '40', 'United-States', '<=50K'], ['67', '?', '212759', '10th', '6', 'Married-civ-spouse', '?', 'Husband', 'White', 'Male', '0', '0', '2', 'United-States', '

[[39,
  'State-gov',
  77516,
  'Bachelors',
  13,
  'Never-married',
  'Adm-clerical',
  'Not-in-family',
  'White',
  'Male',
  2174,
  0,
  40,
  'United-States',
  '<=50K'],
 [50,
  'Self-emp-not-inc',
  83311,
  'Bachelors',
  13,
  'Married-civ-spouse',
  'Exec-managerial',
  'Husband',
  'White',
  'Male',
  0,
  0,
  13,
  'United-States',
  '<=50K'],
 [38,
  'Private',
  215646,
  'HS-grad',
  9,
  'Divorced',
  'Handlers-cleaners',
  'Not-in-family',
  'White',
  'Male',
  0,
  0,
  40,
  'United-States',
  '<=50K'],
 [53,
  'Private',
  234721,
  '11th',
  7,
  'Married-civ-spouse',
  'Handlers-cleaners',
  'Husband',
  'Black',
  'Male',
  0,
  0,
  40,
  'United-States',
  '<=50K'],
 [28,
  'Private',
  338409,
  'Bachelors',
  13,
  'Married-civ-spouse',
  'Prof-specialty',
  'Wife',
  'Black',
  'Female',
  0,
  0,
  40,
  'Cuba',
  '<=50K']]

In [59]:
schema = StructType([
    StructField("age", IntegerType(), True),
    StructField("workclass", StringType(), True),
    StructField("fnlwgt", IntegerType(), True),
    StructField("education", StringType(), True),
    StructField("education_num", IntegerType(), True),
    StructField("marital_status", StringType(), True),
    StructField("occupation", StringType(), True),
    StructField("relationship", StringType(), True),
    StructField("race", StringType(), True),
    StructField("sex", StringType(), True),
    StructField("capital_gain", IntegerType(), True),
    StructField("capital_loss", IntegerType(), True),
    StructField("hours_per_week", IntegerType(), True),
    StructField("native_country", StringType(), True),
    StructField("income", StringType(), True)
])
df_from_rdd = spark.createDataFrame(rdd, schema)
df_from_rdd.show(5)
df_from_rdd.select([F.count(F.col(c)).alias(f"{c}_length") for c in df_from_rdd.columns]).show()

+---+----------------+------+---------+-------------+------------------+-----------------+-------------+-----+------+------------+------------+--------------+--------------+------+
|age|       workclass|fnlwgt|education|education_num|    marital_status|       occupation| relationship| race|   sex|capital_gain|capital_loss|hours_per_week|native_country|income|
+---+----------------+------+---------+-------------+------------------+-----------------+-------------+-----+------+------------+------------+--------------+--------------+------+
| 39|       State-gov| 77516|Bachelors|           13|     Never-married|     Adm-clerical|Not-in-family|White|  Male|        2174|           0|            40| United-States| <=50K|
| 50|Self-emp-not-inc| 83311|Bachelors|           13|Married-civ-spouse|  Exec-managerial|      Husband|White|  Male|           0|           0|            13| United-States| <=50K|
| 38|         Private|215646|  HS-grad|            9|          Divorced|Handlers-cleaners|Not-i

24/11/11 14:56:14 ERROR Executor: Exception in task 1.0 in stage 298.0 (TID 262)
org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/home/fynn/.local/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 1247, in main
    process()
  File "/home/fynn/.local/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 1239, in process
    serializer.dump_stream(out_iter, outfile)
  File "/home/fynn/.local/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/serializers.py", line 274, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/home/fynn/.local/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/util.py", line 83, in wrapper
    return f(*args, **kwargs)
  File "/home/fynn/.local/lib/python3.10/site-packages/pyspark/sql/session.py", line 1459, in prepare
    verify_func(obj)
  File "/home/fynn/.local/lib/python3.10/site-packages/pyspark/s

Py4JJavaError: An error occurred while calling o1311.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 1 in stage 298.0 failed 1 times, most recent failure: Lost task 1.0 in stage 298.0 (TID 262) (129.206.143.174 executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/home/fynn/.local/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 1247, in main
    process()
  File "/home/fynn/.local/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 1239, in process
    serializer.dump_stream(out_iter, outfile)
  File "/home/fynn/.local/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/serializers.py", line 274, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/home/fynn/.local/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/util.py", line 83, in wrapper
    return f(*args, **kwargs)
 

In [41]:
# Read the CSV file
schema = StructType([
    StructField("age", IntegerType(), True),
    StructField("workclass", StringType(), True),
    StructField("fnlwgt", IntegerType(), True),
    StructField("education", StringType(), True),
    StructField("education_num", IntegerType(), True),
    StructField("marital_status", StringType(), True),
    StructField("occupation", StringType(), True),
    StructField("relationship", StringType(), True),
    StructField("race", StringType(), True),
    StructField("sex", StringType(), True),
    StructField("capital_gain", IntegerType(), True),
    StructField("capital_loss", IntegerType(), True),
    StructField("hours_per_week", IntegerType(), True),
    StructField("native_country", StringType(), True),
    StructField("income", StringType(), True)
])
df = spark.read.csv(file_path, header=False, inferSchema=False)



df.show(1000)
df.count()
df.printSchema()


df_cleaned = df.select([F.trim(F.col(c)).alias(c) if df.schema[c].dataType == "StringType" else F.col(c) for c in df.columns])
df_test = df_cleaned.filter(df.workclass == ' ?').show()
test = df.select("workclass").distinct()

distinct_values_list = [row['workclass'] for row in test.collect()]

print(distinct_values_list)

PySparkTypeError: [NOT_LIST_OF_STR] Argument `cols` should be a list[str], got StructType.

## Load Data

In [32]:
adult_csv_rdd = sc.textFile(file_path)

adult_csv_rdd = adult_csv_rdd.map(lambda line: line.split(","))

header = ["age", "workclass", "fnlwgt", "education", "education_num", "marital_status", "occupation", "relationship", "race", "sex", "capital_gain", "capital_loss", "hours_per_week", "native_country", "income"]
df = adult_csv_rdd.toDF(header)

columns_to_cast = ["age", "fnlwgt", "education_num", "capital_gain", "capital_loss", "hours_per_week"]
for column in columns_to_cast:
    df = df.withColumn(column, F.col(column).cast("int"))

df.show(100)
df.printSchema()
print(df.count())

+---+-----------------+------+-------------+-------------+--------------------+------------------+---------------+-------------------+-------+------------+------------+--------------+--------------+------+
|age|        workclass|fnlwgt|    education|education_num|      marital_status|        occupation|   relationship|               race|    sex|capital_gain|capital_loss|hours_per_week|native_country|income|
+---+-----------------+------+-------------+-------------+--------------------+------------------+---------------+-------------------+-------+------------+------------+--------------+--------------+------+
| 39|        State-gov| 77516|    Bachelors|           13|       Never-married|      Adm-clerical|  Not-in-family|              White|   Male|        2174|           0|            40| United-States| <=50K|
| 50| Self-emp-not-inc| 83311|    Bachelors|           13|  Married-civ-spouse|   Exec-managerial|        Husband|              White|   Male|           0|           0|        

24/11/11 12:16:07 ERROR Executor: Exception in task 1.0 in stage 46.0 (TID 54)
java.lang.IllegalStateException: Input row doesn't have expected number of values required by the schema. 15 fields are required while 1 values are provided.
	at org.apache.spark.sql.execution.python.EvaluatePython$$anonfun$$nestedInanonfun$makeFromJava$16$1.applyOrElse(EvaluatePython.scala:188)
	at org.apache.spark.sql.execution.python.EvaluatePython$.nullSafeConvert(EvaluatePython.scala:213)
	at org.apache.spark.sql.execution.python.EvaluatePython$.$anonfun$makeFromJava$16(EvaluatePython.scala:182)
	at org.apache.spark.sql.SparkSession.$anonfun$applySchemaToPythonRDD$2(SparkSession.scala:875)
	at scala.collection.Iterator$$anon$10.next(Iterator.scala:461)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.hashAgg_doAggregateWithoutKey_0$(Unknown Source)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.processNext(Unkn

Py4JJavaError: An error occurred while calling o931.count.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 1 in stage 46.0 failed 1 times, most recent failure: Lost task 1.0 in stage 46.0 (TID 54) (129.206.143.174 executor driver): java.lang.IllegalStateException: Input row doesn't have expected number of values required by the schema. 15 fields are required while 1 values are provided.
	at org.apache.spark.sql.execution.python.EvaluatePython$$anonfun$$nestedInanonfun$makeFromJava$16$1.applyOrElse(EvaluatePython.scala:188)
	at org.apache.spark.sql.execution.python.EvaluatePython$.nullSafeConvert(EvaluatePython.scala:213)
	at org.apache.spark.sql.execution.python.EvaluatePython$.$anonfun$makeFromJava$16(EvaluatePython.scala:182)
	at org.apache.spark.sql.SparkSession.$anonfun$applySchemaToPythonRDD$2(SparkSession.scala:875)
	at scala.collection.Iterator$$anon$10.next(Iterator.scala:461)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIterat

In [20]:
print(df.count())

24/11/11 11:45:26 ERROR Executor: Exception in task 1.0 in stage 27.0 (TID 32)
java.lang.IllegalStateException: Input row doesn't have expected number of values required by the schema. 15 fields are required while 1 values are provided.
	at org.apache.spark.sql.execution.python.EvaluatePython$$anonfun$$nestedInanonfun$makeFromJava$16$1.applyOrElse(EvaluatePython.scala:188)
	at org.apache.spark.sql.execution.python.EvaluatePython$.nullSafeConvert(EvaluatePython.scala:213)
	at org.apache.spark.sql.execution.python.EvaluatePython$.$anonfun$makeFromJava$16(EvaluatePython.scala:182)
	at org.apache.spark.sql.SparkSession.$anonfun$applySchemaToPythonRDD$2(SparkSession.scala:875)
	at scala.collection.Iterator$$anon$10.next(Iterator.scala:461)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.hashAgg_doAggregateWithoutKey_0$(Unknown Source)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.processNext(Unkn

Py4JJavaError: An error occurred while calling o432.count.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 1 in stage 27.0 failed 1 times, most recent failure: Lost task 1.0 in stage 27.0 (TID 32) (129.206.143.174 executor driver): java.lang.IllegalStateException: Input row doesn't have expected number of values required by the schema. 15 fields are required while 1 values are provided.
	at org.apache.spark.sql.execution.python.EvaluatePython$$anonfun$$nestedInanonfun$makeFromJava$16$1.applyOrElse(EvaluatePython.scala:188)
	at org.apache.spark.sql.execution.python.EvaluatePython$.nullSafeConvert(EvaluatePython.scala:213)
	at org.apache.spark.sql.execution.python.EvaluatePython$.$anonfun$makeFromJava$16(EvaluatePython.scala:182)
	at org.apache.spark.sql.SparkSession.$anonfun$applySchemaToPythonRDD$2(SparkSession.scala:875)
	at scala.collection.Iterator$$anon$10.next(Iterator.scala:461)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIterat

In [10]:
#b)
window_marital_status = Window.partitionBy("marital_status")

# count the number of rows using the window
df_with_count = df.withColumn("count",
            F.count("marital_status").over(window_marital_status)
)
df_with_count.show(10)

#add a new column using the window to get the percentage of males in each marital_status group
# df.withColumn("Male Percentage",
#             F.count(
#                 df.filter(df.relationship == "Husband")).over(window_marital_status)
# )


24/11/11 11:27:13 WARN DiskBlockObjectWriter: Error deleting /tmp/blockmgr-18413ac9-1987-4150-bc2a-f87d76288408/37/temp_shuffle_d2999fda-34a4-4324-93c4-93b0e1d725d8
24/11/11 11:27:13 WARN DiskBlockObjectWriter: Error deleting /tmp/blockmgr-18413ac9-1987-4150-bc2a-f87d76288408/24/temp_shuffle_8f80766f-d251-4c3f-b55e-ebc6e7f377b4
24/11/11 11:27:13 WARN DiskBlockObjectWriter: Error deleting /tmp/blockmgr-18413ac9-1987-4150-bc2a-f87d76288408/01/temp_shuffle_eaf935a0-bf3a-4d13-8715-9ad76b1af99b
24/11/11 11:27:13 WARN DiskBlockObjectWriter: Error deleting /tmp/blockmgr-18413ac9-1987-4150-bc2a-f87d76288408/22/temp_shuffle_d6b25f31-0891-410e-aa1f-9e70d3995b75
24/11/11 11:27:13 WARN DiskBlockObjectWriter: Error deleting /tmp/blockmgr-18413ac9-1987-4150-bc2a-f87d76288408/24/temp_shuffle_b54789c2-8812-433b-945a-cc9cdaa4d133
24/11/11 11:27:13 WARN DiskBlockObjectWriter: Error deleting /tmp/blockmgr-18413ac9-1987-4150-bc2a-f87d76288408/16/temp_shuffle_277252d8-e8e7-4a99-9eae-d99372de4757
24/11/11 1

Py4JJavaError: An error occurred while calling o165.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 1 in stage 9.0 failed 1 times, most recent failure: Lost task 1.0 in stage 9.0 (TID 12) (129.206.143.174 executor driver): java.lang.IllegalStateException: Input row doesn't have expected number of values required by the schema. 15 fields are required while 1 values are provided.
	at org.apache.spark.sql.execution.python.EvaluatePython$$anonfun$$nestedInanonfun$makeFromJava$16$1.applyOrElse(EvaluatePython.scala:188)
	at org.apache.spark.sql.execution.python.EvaluatePython$.nullSafeConvert(EvaluatePython.scala:213)
	at org.apache.spark.sql.execution.python.EvaluatePython$.$anonfun$makeFromJava$16(EvaluatePython.scala:182)
	at org.apache.spark.sql.SparkSession.$anonfun$applySchemaToPythonRDD$2(SparkSession.scala:875)
	at scala.collection.Iterator$$anon$10.next(Iterator.scala:461)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIte